## On souhaite rajouter des données d'Open data à notre jeu de données, on va le faire via le code postal et en même temps rajouter les colonnes lat et lon ainsi que température(open data)

#### Lecture de nos fichiers csv

In [4]:
import pandas as pd

df_postal = pd.read_csv("../data/adresses-69.csv", sep=";")
list(df_postal.columns)

C:\Users\quent\AppData\Local\Temp\ipykernel_22844\1030823535.py:3: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_postal = pd.read_csv("../data/adresses-69.csv", sep=";")


['id',
 'id_fantoir',
 'numero',
 'rep',
 'nom_voie',
 'code_postal',
 'code_insee',
 'nom_commune',
 'code_insee_ancienne_commune',
 'nom_ancienne_commune',
 'x',
 'y',
 'lon',
 'lat',
 'type_position',
 'alias',
 'nom_ld',
 'libelle_acheminement',
 'nom_afnor',
 'source_position',
 'source_nom_voie',
 'certification_commune',
 'cad_parcelles']

In [6]:
df_merged = pd.read_csv("../data/dataset_M2_enedis.csv", sep=";")


C:\Users\quent\AppData\Local\Temp\ipykernel_22844\1718932970.py:1: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_merged = pd.read_csv("../data/dataset_M2_enedis.csv", sep=";")


In [23]:
list(df_merged.columns)


['Unnamed: 0',
 'Ubat_W/m²_K',
 "Complément_d'adresse_bâtiment",
 'Méthode_application_DPE',
 'N°_département_(BAN)',
 'Qualité_isolation_menuiseries',
 'Nombre_appartement',
 'Emission_GES_5_usages_énergie_n°2',
 'Coordonnée_cartographique_X_(BAN)',
 'Date_fin_validité_DPE',
 'Protection_solaire_exterieure_(0/1)',
 'Coût_total_5_usages',
 'Conso_ECS_dépensier_é_primaire',
 'Catégorie_ENR',
 'Présence_brasseur_air_(0/1)',
 'Date_établissement_DPE',
 'Score_BAN',
 'Type_énergie_n°2',
 'Conso_auxiliaires_é_primaire',
 'N°_étage_appartement',
 'N°_région_(BAN)',
 'Deperditions_planchers_hauts',
 'Adresse_brute',
 'Emission_GES_refroidissement_dépensier',
 'Version_DPE',
 'Deperditions_planchers_bas',
 'Surface_tertiaire_immeuble',
 'Coût_chauffage',
 '_i',
 'Qualité_isolation_plancher_bas',
 'Déperditions_renouvellement_air',
 'Surface_habitable_logement',
 'Type_énergie_n°1',
 'Conso_chauffage_é_finale',
 'Besoin_refroidissement',
 'Emission_GES_éclairage',
 'Code_INSEE_(BAN)',
 'Conso_r

### Jointure données OPEN DATA

In [8]:
df_merged['_geopoint']

0        46.00165800390864,4.718454032425727
1        45.75699900642668,4.858042958764985
2        45.76897500501959,4.890034987692009
3        45.73898902628077,4.770525958839136
4        45.76291503709679,4.852469023561155
                        ...                 
9995    45.799381959479945,5.043622052454483
9996    45.764759014549035,4.857382950344493
9997     45.67805201380173,4.861176969838147
9998      45.7140430178337,4.878583053771784
9999     45.79153296099185,4.864253030712091
Name: _geopoint, Length: 10000, dtype: object

### Données Socio-economique de l'INSEE

https://portail-api.insee.fr/

In [6]:
# import requests
# import json

# url = "https://api.insee.fr/melodi/catalog/DERA"
# response = requests.get(url)



**En explorant les données de l'INSEE, on se rend compte que l'INSEE ne fournit pas de données sur les adresses précises (comme les coordonnées latitude/longitude exactes) des domiciles individuels dans ses jeux de données**, on ne peut donc pas faire un regroupement pour chaque domicile mais on peut avoir des données selon les codes postaux.

**On va devoir se contenter de données socio-économiques sur les codes postaux**

In [7]:
# import requests
# import json

# url = "https://api.insee.fr/melodi/data/DS_RP_POPULATION_PRINC?maxResult=200&GEO=2024-ARM-69381&GEO=2024-ARM-69382&GEO=2024-AAV2020-002&GEO=DEP"
# response = requests.get(url)
# # print(response._content)
# data = json.loads(response._content)
# data
# print(len(data))

# # Vu que les données qui nous intéressent sont sensibles et ne sont pas disponible,
# # on a décidé de juste prendre une donnée plus simple comme la température de la ville de Lyon


#### Import de la température via API
La température de la ville de Lyon par département est une série temporelle, on souhaite faire une comparaison des température des départements du rhône, on va donc prendre la moyenne des températures de l'année 2023 afin de comparer les départements entre eux

https://www.infoclimat.fr/fr/cartes/observations-meteo/temps-reel/temperature/france_ce.html

pour le 69, les stations dispose de lat et lon et des noms des communes, le fichier adresse-69 contient le nom des communes, on pourrait joindres les données sur ça

https://www.infoclimat.fr/opendata/?version=2&method=get&format=json&stations[]=ME128&stations[]=ME046&stations[]=000BG&stations[]=STATIC0007&stations[]=07481&stations[]=000GM&stations[]=STATIC0315&stations[]=STATIC0139&stations[]=000C9&stations[]=STATIC0065&stations[]=000SP&stations[]=STATIC0068&start=2024-11-07&end=2024-11-09&token=oFoV74U77pPDxguuxL5Hg5jahlu8wwo9HBv5qXzpzgO8L1LqOQ

**On peut aussi télécharger en fichier csv, c'est plus simple car pour faire une requête API, il faut un jeton qui ne dure que 7 jours, on va donc utiliser l'export en csv**

In [9]:
import pandas as pd
df_temp_rhone = pd.read_csv("../data/export_infoclimat.csv", sep=";")
# api_key = "oFoV74U77pPDxguuxL5Hg5jahlu8wwo9HBv5qXzpzgO8L1LqOQ"	
# url="https://www.infoclimat.fr/opendata/?version=2&method=get&format=json&stations[]=ME128&stations[]=ME046&stations[]=000BG&stations[]=STATIC0007&stations[]=07481&stations[]=000GM&stations[]=STATIC0315&stations[]=STATIC0139&stations[]=000C9&stations[]=STATIC0065&stations[]=000SP&stations[]=STATIC0068&start=2024-11-07&end=2024-11-09&token=oFoV74U77pPDxguuxL5Hg5jahlu8wwo9HBv5qXzpzgO8L1LqOQ"

In [9]:
# response = requests.get(url)
# print(response)
# print(response._content)

In [10]:
df_temp_rhone.head()

# METADATA:
# 000BG: ENS - Lyon 7Ã¨me
# 000C9: Sain-Bel
# 000GM: Yzeron
# 000SP: Montagny
# 07481: Lyon-St ExupÃ©ry
# ME046: [MAE] Ecole Normale SupÃ©rieure - LYON
# ME128: [MAE] CollÃ¨ge HonorÃ© De Balzac - VENISSIEUX
# STATIC0007: Corbas
# STATIC0065: Soucieu-en-Jarrest
# STATIC0139: Brullioles
# STATIC0315: Saint-AndrÃ©-la-CÃ´te
# Source: www.infoclimat.fr/MeteoFrance
# Licence Ouverte/Open License: http://wiki.data.gouv.fr/images/9/9d/Licence_Ouverte.pdf


,station_id,dh_utc,temperature,pression,pression_variation_3h,humidite,point_de_rosee,visibilite,vent_moyen,vent_rafales,...,radiations,neige_au_sol,nebulosite,raw_msg,pluie_cumul_0h,pluie_intensite,pluie_intensite_max_1h,uv,uv_index,complements
0,000BG,07/11/2024 00:00,13.1,1027.7,NaN,93.0,12.2,NaN,1.6,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0@@0@@1037.1@@@11.3@23.1@23.1@23.2@12:39am@12:...
1,000BG,07/11/2024 00:10,13.1,1027.6,NaN,93.0,12.2,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0@@0@@1037.1@@@11.3@23.1@23.1@23.2@1:15am@12:0...
2,000BG,07/11/2024 00:20,13.0,1027.5,NaN,94.0,12.2,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0@@0@@1037.1@@@11.3@23.1@23.1@23.2@1:15am@12:0...
3,000BG,07/11/2024 00:30,13.0,1027.4,NaN,94.0,12.2,NaN,1.6,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0@@0@@1037.1@@@11.3@23.1@23.1@23.2@1:15am@12:0...
4,000BG,07/11/2024 00:40,12.9,1027.4,NaN,94.0,12.2,NaN,1.6,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0@@0@@1037.1@@@11.3@23.1@23@23.2@1:45am@12:00a...


In [11]:
# Print the columns of the dataframe
print(df_temp_rhone.columns)

# Mapping dictionary for station IDs to postal codes
station_id_to_postal_code = {
  '000BG': '69007',
  '000C9': '69210',
  '000GM': '69510',
  '000SP': '69700',
  '07481': '69125',
  'ME046': '69007',
  'ME128': '69200',
  'STATIC0007': '69960',
  'STATIC0065': '69510',
  'STATIC0139': '69690',
  'STATIC0315': '69440'
}

# Rename station IDs by their postal codes
df_temp_rhone['station_id'] = df_temp_rhone['station_id'].replace(station_id_to_postal_code)


Index(['station_id', 'dh_utc', 'temperature', 'pression',
       'pression_variation_3h', 'humidite', 'point_de_rosee', 'visibilite',
       'vent_moyen', 'vent_rafales', 'vent_rafales_10min', 'vent_direction',
       'temperature_min', 'temperature_max', 'pluie_1h', 'pluie_3h',
       'pluie_6h', 'pluie_12h', 'pluie_24h', 'ensoleillement',
       'temperature_sol', 'temps_omm', 'source', 'radiations', 'neige_au_sol',
       'nebulosite', 'raw_msg', 'pluie_cumul_0h', 'pluie_intensite',
       'pluie_intensite_max_1h', 'uv', 'uv_index', 'complements'],
      dtype='object')


In [12]:
print(df_temp_rhone.sample(5))
# On vient bien que les stations sont bien renommées par leur code postal
# On ne va garder que la température et le code postal
df_temp_rhone = df_temp_rhone[['temperature', 'station_id']]
print(df_temp_rhone.head())

     station_id            dh_utc  temperature  pression  \
1308      69125  07/11/2024 01:00         10.9    1028.9   
1459      69200  07/11/2024 17:50         13.6    1024.7   
1230      69700  08/11/2024 22:10          8.0    1025.0   
1292      69700  09/11/2024 08:30          9.8    1025.2   
1523      69200  08/11/2024 06:40          NaN    1023.2   

      pression_variation_3h  humidite  point_de_rosee  visibilite  vent_moyen  \
1308                   -0.5      96.0            10.3      1500.0         3.6   
1459                   -0.1      90.0            12.2         NaN         0.0   
1230                    0.5      94.0             7.0         NaN         0.0   
1292                    0.5      95.0             9.1         NaN         1.0   
1523                   -0.7       NaN             NaN         NaN         NaN   

      vent_rafales  ...  radiations  neige_au_sol  nebulosite  \
1308          10.8  ...         NaN           NaN         8.0   
1459           NaN  ..

In [13]:
# On va maintenant faire la moyenne de la température par code postal
df_temp_rhone = df_temp_rhone.groupby('station_id').mean().reset_index()
df_temp_rhone

,station_id,temperature
0,69007,11.996893
1,69125,10.786441
2,69200,12.195926
3,69210,11.041880
4,69440,10.533616
5,69510,10.850751
6,69690,10.935345
7,69700,11.324859
8,69960,11.430932


In [14]:
print(df_temp_rhone.info()) # station_id de type object, on va convertir en str pour la jointure
df_postal['code_postal'] = df_postal['code_postal'].astype(str)
# On va maintenant faire la jointure avec le dataframe df_total_lon_lat
df_postal = pd.merge(df_postal, df_temp_rhone, left_on='code_postal', right_on='station_id', how='left')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   station_id   9 non-null      object 
 1   temperature  9 non-null      float64
dtypes: float64(1), object(1)
memory usage: 276.0+ bytes
None


In [15]:
# Afficher que les données ayant un code postal égal à 69007 pour voir si la température est bien renseignée
df_postal[df_postal['code_postal'] == '69007']['temperature']

12971    11.996893
12972    11.996893
12973    11.996893
12974    11.996893
12975    11.996893
           ...    
17338    11.996893
17339    11.996893
17340    11.996893
17341    11.996893
17342    11.996893
Name: temperature, Length: 4372, dtype: float64

In [20]:
df_postal.head()

,id,id_fantoir,numero,rep,nom_voie,code_postal,code_insee,nom_commune,code_insee_ancienne_commune,nom_ancienne_commune,...,alias,nom_ld,libelle_acheminement,nom_afnor,source_position,source_nom_voie,certification_commune,cad_parcelles,station_id,temperature
0,69002_x415an_00003,NaN,3,NaN,Route de Matour,69790,69002,Aigueperse,NaN,NaN,...,NaN,NaN,AIGUEPERSE,ROUTE DE MATOUR,commune,commune,1,NaN,NaN,NaN
1,69002_x415an_00111,NaN,111,NaN,Route de Matour,69790,69002,Aigueperse,NaN,NaN,...,NaN,NaN,AIGUEPERSE,ROUTE DE MATOUR,commune,commune,1,NaN,NaN,NaN
2,69002_x415an_00165,NaN,165,NaN,Route de Matour,69790,69002,Aigueperse,NaN,NaN,...,NaN,NaN,AIGUEPERSE,ROUTE DE MATOUR,commune,commune,1,NaN,NaN,NaN
3,69002_x415an_01148,NaN,1148,NaN,Route de Matour,69790,69002,Aigueperse,NaN,NaN,...,NaN,NaN,AIGUEPERSE,ROUTE DE MATOUR,commune,commune,1,NaN,NaN,NaN
4,69002_wit9km_00007,NaN,7,NaN,Chemin des Dalhias,69790,69002,Aigueperse,NaN,NaN,...,NaN,NaN,AIGUEPERSE,CHEMIN DES DALHIAS,commune,commune,1,NaN,NaN,NaN


### Jointure de la longitude, latitude et température(open data) à notre jeu de données:

In [ ]:
# Faire la jointure du fichier adresse_69 qui contient la latitude et longtiude avec les fichiers existant_69 et neuf_69
# Left_on='identifiant_BAN' et right_on = 'id', how = 'left'
df_total_lon_lat_temp = pd.merge(df_merged, df_postal[['lon', 'lat', 'temperature','id']], left_on='Identifiant__BAN', right_on='id', how='left')

**Ici le merge ne marche plus car le fichier df_merged contient déjà les températures, on ne peut pas importer le fichier des données ENEDIS car trop gros donc la suite du notebook n'est pas pertinente à éxecuter, le but est de montrer uniquement le processus de concaténation de données d'Open data et celles de l'ADEME**
refaire un merge donne des clef du type `temperature_y` au lieu de `temperature`, ce qui compromets la suite du notebook

In [21]:
df_total_lon_lat_temp.columns
# On vérifie que la température est bien renseignée pour la ville de Lyon
# df_total_lon_lat_temp['temperature'].isnull().sum()/len(df_total_lon_lat_temp)


Index(['Unnamed: 0', 'Ubat_W/m²_K', 'Complément_d'adresse_bâtiment',
       'Méthode_application_DPE', 'N°_département_(BAN)',
       'Qualité_isolation_menuiseries', 'Nombre_appartement',
       'Emission_GES_5_usages_énergie_n°2',
       'Coordonnée_cartographique_X_(BAN)', 'Date_fin_validité_DPE',
       ...
       'Appartement_non_visité_(0/1)', 'Complément_d'adresse_logement',
       'lon_x', 'lat_x', 'temperature_x', 'id_x', 'lon_y', 'lat_y',
       'temperature_y', 'id_y'],
      dtype='object', length=140)

In [18]:
# Malheureusement, l'API de la température est limité, on n'a pas pu avoir la température pour toutes les communes du rhône mais seulement quelques unes
# L'information n'est donc sûrement pas pertinentes et va sûrement entre enlever dans le préprocessing des données

In [ ]:
# export du csv
df_total_lon_lat_temp.to_csv("../data/df_total_lon_lat_temp.csv", sep=";")

In [ ]:
# Le fichier est trop lourd pour git, on a exploré git LFS mais nous avons choisi de juste réduire la taille car on a pas besoin d'autant de données pour l'exploration
# On va donc réduire la taille du fichier en prenant un sample de 10_000 lignes

df_total_lon_lat_temp_sample = df_total_lon_lat_temp.sample(10_000)
df_total_lon_lat_temp_sample.to_csv("../data/dataset_M2_enedis.csv", sep=";")

In [ ]:
# On update le fichier preprocessed_data.py pour ajouter la température

from pre_processing import preprocess_data

df_classification_preprocessesed = preprocess_data(df_total_lon_lat_temp, sauvegarde=True)


Prétraitement des données...
Removing missing values...
Encodage des variables catégorielles...
Gestion des outliers...
Gestion des valeurs manquantes...
Export des données...
Export réussi sous le nom de 'preprocessed_data.csv'


In [27]:
list(df_classification_preprocessesed.columns)

['Ubat_W/m²_K',
 'Méthode_application_DPE',
 'N°_département_(BAN)',
 'Qualité_isolation_menuiseries',
 'Nombre_appartement',
 'Coordonnée_cartographique_X_(BAN)',
 'Date_fin_validité_DPE',
 'Coût_total_5_usages',
 'Conso_ECS_dépensier_é_primaire',
 'Date_établissement_DPE',
 'Score_BAN',
 'Conso_auxiliaires_é_primaire',
 'N°_étage_appartement',
 'N°_région_(BAN)',
 'Deperditions_planchers_hauts',
 'Adresse_brute',
 'Emission_GES_refroidissement_dépensier',
 'Version_DPE',
 'Deperditions_planchers_bas',
 'Coût_chauffage',
 '_i',
 'Qualité_isolation_plancher_bas',
 'Déperditions_renouvellement_air',
 'Surface_habitable_logement',
 'Type_énergie_n°1',
 'Conso_chauffage_é_finale',
 'Besoin_refroidissement',
 'Emission_GES_éclairage',
 'Code_INSEE_(BAN)',
 'Conso_refroidissement_dépensier_é_primaire',
 'Conso_ECS_dépensier_é_finale',
 '_score',
 'Classe_altitude',
 'Coût_ECS',
 'Conso_ECS_é_primaire',
 'Nombre_niveau_logement',
 'Adresse_(BAN)',
 'Emission_GES_ECS',
 'Conso_chauffage_dépen

In [30]:
# Le fichier est trop lourd, je vais donc garder que les colonnes utiliser pour l'entraînement du modèle
# 'Conso5_usages_par_m²éprimaire',
#  'Conso_5_usages/m²éfinale',
#  'Emission_GES_5_usages_par_m²',
#  'Etiquette_GES',
#  'Coûtéclairage']
preprocess_data_classification = df_classification_preprocessesed[['Conso_5_usages_par_m²_é_primaire', 'Conso_5_usages/m²_é_finale', 'Emission_GES_5_usages_par_m²', 'Etiquette_GES', 'Coût_éclairage']]

In [ ]:
# export du fichier qui va être utilisé pour l'entraînement du modèle
preprocess_data_classification.to_csv("../data/preprocess_data_classification.csv", sep=";")